In [1]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sklearn
# Importing the dataset
alldata=pd.read_csv('Strawberry_data_1_6.csv')
X = alldata.iloc[:, :].values
y = alldata.iloc[:, 224].values
X = alldata.iloc[:, 1:223].values
print(X,y)

[[1.8099  1.8784  1.9487  ... 1.0771  1.063   1.0477 ]
 [1.8042  1.8663  1.9334  ... 0.93248 0.91894 0.90425]
 [1.5693  1.6249  1.682   ... 0.81687 0.80471 0.79143]
 ...
 [2.8768  2.898   2.921   ... 0.77666 0.77487 0.77302]
 [2.7546  2.7826  2.8058  ... 0.60506 0.60352 0.6019 ]
 [3.1097  3.1569  3.2092  ... 0.78538 0.7833  0.78087]] [1 1 1 ... 6 6 6]


In [2]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0,stratify = y)

In [3]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

In [4]:
from sklearn.feature_selection import VarianceThreshold
sel = VarianceThreshold(threshold=(.8 * (1 - .8)))
sel.fit_transform(X)

array([[1.8099 , 1.8784 , 1.9487 , ..., 1.6885 , 1.6112 , 1.5394 ],
       [1.8042 , 1.8663 , 1.9334 , ..., 1.7936 , 1.7    , 1.6106 ],
       [1.5693 , 1.6249 , 1.682  , ..., 1.0665 , 1.0175 , 0.97257],
       ...,
       [2.8768 , 2.898  , 2.921  , ..., 2.5483 , 2.5202 , 2.4907 ],
       [2.7546 , 2.7826 , 2.8058 , ..., 2.3703 , 2.3512 , 2.3306 ],
       [3.1097 , 3.1569 , 3.2092 , ..., 3.122  , 3.0611 , 2.9983 ]])

# SVM Model

In [28]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
# Generate random dataset
X, y = make_classification(n_samples=100, n_features=10, random_state=42)

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Feature scaling
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


# Create and train the SVM classifier
svm = SVC(kernel = 'poly', random_state = 0)
svm.fit(X_train, y_train)

# Predict on the test data
y_pred = svm.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 1.0


# Derivative-based feature selection + SVM

In [7]:
import numpy as np
from sklearn.svm import SVC
from sklearn.datasets import make_classification
from scipy.stats import pearsonr

# Generate random dataset
X, y = make_classification(n_samples=100, n_features=10, random_state=42)

# Compute Pearson correlation coefficients for each feature
correlations = []
for i in range(X.shape[1]):
    corr, _ = pearsonr(X[:, i], y)
    correlations.append(abs(corr))

# Sort features based on correlation coefficient
sorted_indices = np.argsort(correlations)[::-1]
sorted_X = X[:, sorted_indices]

# Select top k features
k = 10
selected_features = sorted_X[:, :k]

# Split data into train and test sets
split_ratio = 0.8
split_index = int(X.shape[0] * split_ratio)
X_train, X_test = selected_features[:split_index], selected_features[split_index:]
y_train, y_test = y[:split_index], y[split_index:]

# Train SVM classifier
svm = SVC()
svm.fit(X_train, y_train)

# Predict on test data
y_pred = svm.predict(X_test)

# Evaluate performance
accuracy = np.mean(y_pred == y_test)
print("Accuracy:", accuracy)


Accuracy: 0.95


# Derivative-based feature selection + ANN

In [8]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras import models, layers
from scipy.stats import pearsonr

# Generate random dataset
X, y = make_classification(n_samples=100, n_features=10, random_state=42)

# Compute Pearson correlation coefficients for each feature
correlations = []
for i in range(X.shape[1]):
    corr, _ = pearsonr(X[:, i], y)
    correlations.append(abs(corr))

# Sort features based on correlation coefficient
sorted_indices = np.argsort(correlations)[::-1]
sorted_X = X[:, sorted_indices]

# Select top k features
k = 5
selected_features = sorted_X[:, :k]

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(selected_features, y, test_size=0.2, random_state=42)

# Feature scaling
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Create the ANN model
model = models.Sequential()
model.add(layers.Dense(16, activation='relu', input_dim=k))
model.add(layers.Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=0)

# Evaluate the model
_, accuracy = model.evaluate(X_test, y_test)
print("Accuracy:", accuracy)


1/1 [==============================] - 0s 360ms/step - loss: 0.1118 - accuracy: 1.0000
Accuracy: 1.0


In [9]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from scipy.stats import pearsonr

# Generate random dataset
X, y = make_classification(n_samples=100, n_features=10, random_state=42)

# Compute Pearson correlation coefficients for each feature
correlations = []
for i in range(X.shape[1]):
    corr, _ = pearsonr(X[:, i], y)
    correlations.append(abs(corr))

# Sort features based on correlation coefficient
sorted_indices = np.argsort(correlations)[::-1]
sorted_X = X[:, sorted_indices]

# Select top k features
k = 5
selected_features = sorted_X[:, :k]

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(selected_features, y, test_size=0.2, random_state=42)

# Feature scaling
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Create and train the KNN classifier
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train, y_train)

# Predict on test data
y_pred = knn.predict(X_test)

# Evaluate the classifier
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

accuracy=sklearn.metrics.accuracy_score(y_test,y_pred)
print(accuracy)




1.0


C:\Users\Aqeel\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


In [10]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from scipy.stats import pearsonr

# Generate random dataset
X, y = make_classification(n_samples=100, n_features=10, random_state=42)

# Compute Pearson correlation coefficients for each feature
correlations = []
for i in range(X.shape[1]):
    corr, _ = pearsonr(X[:, i], y)
    correlations.append(abs(corr))

# Sort features based on correlation coefficient
sorted_indices = np.argsort(correlations)[::-1]
sorted_X = X[:, sorted_indices]

# Select top k features
k = 5
selected_features = sorted_X[:, :k]

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(selected_features, y, test_size=0.2, random_state=42)

# Feature scaling
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Create and train the Random Forest classifier
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# Predict on test data
y_pred = rf.predict(X_test)

# Evaluate the classifier
accuracy = np.mean(y_pred == y_test)
print("Accuracy:", accuracy)


Accuracy: 1.0


In [11]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from scipy.stats import pearsonr

# Generate random dataset
X, y = make_classification(n_samples=100, n_features=10, random_state=42)

# Compute Pearson correlation coefficients for each feature
correlations = []
for i in range(X.shape[1]):
    corr, _ = pearsonr(X[:, i], y)
    correlations.append(abs(corr))

# Sort features based on correlation coefficient
sorted_indices = np.argsort(correlations)[::-1]
sorted_X = X[:, sorted_indices]

# Select top k features
k = 5
selected_features = sorted_X[:, :k]

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(selected_features, y, test_size=0.2, random_state=42)

# Feature scaling
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Create and train the Decision Tree classifier
dt = DecisionTreeClassifier(random_state=42)
dt.fit(X_train, y_train)

# Predict on test data
y_pred = dt.predict(X_test)

# Evaluate the classifier
accuracy = np.mean(y_pred == y_test)
print("Accuracy:", accuracy)


Accuracy: 0.95


In [12]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.cross_decomposition import PLSRegression
from sklearn.metrics import accuracy_score

# Generate random dataset
X, y = make_classification(n_samples=100, n_features=10, random_state=42)

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Feature scaling
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Create and train the PLS-DA model
plsda = PLSRegression(n_components=2)
plsda.fit(X_train, y_train)

# Transform the data
X_train_transformed = plsda.transform(X_train)
X_test_transformed = plsda.transform(X_test)

# Predict the labels
y_pred_train = np.argmax(X_train_transformed, axis=1)
y_pred_test = np.argmax(X_test_transformed, axis=1)

# Calculate accuracy
train_accuracy = accuracy_score(y_train, y_pred_train)
test_accuracy = accuracy_score(y_test, y_pred_test)

print("Train Accuracy:", train_accuracy)
print("Test Accuracy:", test_accuracy)


Train Accuracy: 0.2375
Test Accuracy: 0.45


In [13]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Generate random dataset
X, y = make_classification(n_samples=100, n_features=10, random_state=42)

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Feature scaling
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Apply PCA for dimensionality reduction
pca = PCA(n_components=5)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

# Train SVM classifier on the reduced data
svm = SVC()
svm.fit(X_train_pca, y_train)

# Predict on the test data
y_pred = svm.predict(X_test_pca)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.95


In [14]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import accuracy_score

# Generate random dataset
X, y = make_classification(n_samples=100, n_features=10, random_state=42)

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Feature scaling
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Create and train the LDA model
lda = LinearDiscriminantAnalysis()
X_train_lda = lda.fit_transform(X_train, y_train)
X_test_lda = lda.transform(X_test)

# Train SVM classifier on the reduced data
svm = SVC()
svm.fit(X_train_lda, y_train)

# Predict on the test data
y_pred = svm.predict(X_test_lda)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 1.0


In [16]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

accuracy=sklearn.metrics.accuracy_score(y_test,y_pred)
print(accuracy)


1.0


In [19]:
# Cross Validation
from sklearn import model_selection
from sklearn.model_selection import cross_val_score
kfold = model_selection.KFold(n_splits=10)
modelCV = LinearDiscriminantAnalysis()
scoring = 'accuracy'
results = model_selection.cross_val_score(modelCV, X_train, y_train, cv=kfold, scoring=scoring)
print("10-fold cross validation average accuracy: %.3f" % (results.mean()))

10-fold cross validation average accuracy: 0.938


In [22]:
print("10-fold cross validation average accuracy: %.3f" % (results.mean()))
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_test, y_pred)
print(confusion_matrix)


from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))


10-fold cross validation average accuracy: 0.938
[[ 9  0]
 [ 0 11]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         9
           1       1.00      1.00      1.00        11

    accuracy                           1.00        20
   macro avg       1.00      1.00      1.00        20
weighted avg       1.00      1.00      1.00        20



# CNN Model

In [23]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# Split the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create an ANN model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32)

# Make predictions on the test data
y_pred = model.predict(X_test)
y_pred_labels = np.round(y_pred).flatten()

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred_labels)
print("Accuracy:", accuracy)

# Generate classification report
report = classification_report(y_test, y_pred_labels)
print("Classification Report:")
print(report)


Epoch 1/10
3/3 [==============================] - 1s 7ms/step - loss: 0.6784 - accuracy: 0.5625
Epoch 2/10
3/3 [==============================] - 0s 4ms/step - loss: 0.6298 - accuracy: 0.6500
Epoch 3/10
3/3 [==============================] - 0s 5ms/step - loss: 0.5862 - accuracy: 0.7750
Epoch 4/10
3/3 [==============================] - 0s 5ms/step - loss: 0.5479 - accuracy: 0.8750
Epoch 5/10
3/3 [==============================] - 0s 5ms/step - loss: 0.5108 - accuracy: 0.9625
Epoch 6/10
3/3 [==============================] - 0s 4ms/step - loss: 0.4744 - accuracy: 0.9625
Epoch 7/10
3/3 [==============================] - 0s 4ms/step - loss: 0.4402 - accuracy: 0.9625
Epoch 8/10
3/3 [==============================] - 0s 7ms/step - loss: 0.4061 - accuracy: 0.9625
Epoch 9/10
3/3 [==============================] - 0s 7ms/step - loss: 0.3736 - accuracy: 0.9875
Epoch 10/10
1/1 [==============================] - 0s 87ms/step
Accuracy: 1.0
Classification Report:
              precision    recall  

In [27]:
# see how the training went
from matplotlib import pyplot as plt
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['Train', 'Cross-Validation'], loc='upper left')
plt.show()

NameError: name 'history' is not defined

In [ ]:
#loss function
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('epoch')
plt.legend(['Train', 'Cross-Validation'], loc='upper left')
plt.show()